# test_splinter_scrape_ta_reviews

In [19]:
from splinter.browser import Browser
import connect_aws_db as cadb
from splinter_scrape_ta_reviews import get_hotel_urls
from splinter_scrape_ta_reviews import scrape_hotel
from splinter_scrape_ta_reviews import get_done_business_ids
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
#import sys  
#reload(sys)  
#sys.setdefaultencoding('utf8')

In [5]:
city = 'new_haven'
state = 'ct'

In [6]:
engine = cadb.connect_aws_db(write_unicode=True)

In [7]:
blinks = get_hotel_urls(city, state, engine)

In [8]:
br = Browser()

In [8]:
for hotel_id, biz_id, link in blinks:
    print(hotel_id, biz_id, link)

(3278, u'1217689', u'http://www.tripadvisor.com/Hotel_Review-g33851-d1217689-Reviews-The_Study_at_Yale-New_Haven_Connecticut.html')
(3279, u'85971', u'http://www.tripadvisor.com/Hotel_Review-g33851-d85971-Reviews-La_Quinta_Inn_Suites_New_Haven-New_Haven_Connecticut.html')
(3280, u'225916', u'http://www.tripadvisor.com/Hotel_Review-g33851-d225916-Reviews-Days_Inn_New_Haven-New_Haven_Connecticut.html')
(3281, u'119885', u'http://www.tripadvisor.com/Hotel_Review-g33851-d119885-Reviews-Duncan_Hotel-New_Haven_Connecticut.html')
(3282, u'119904', u'http://www.tripadvisor.com/Hotel_Review-g33851-d119904-Reviews-Atlantic_Motel-New_Haven_Connecticut.html')
(3283, u'83702', u'http://www.tripadvisor.com/Hotel_Review-g33851-d83702-Reviews-Regal_Inn-New_Haven_Connecticut.html')
(3284, u'119851', u'http://www.tripadvisor.com/Hotel_Review-g33851-d119851-Reviews-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html')
(3285, u'85912', u'http://www.tripadvisor.com/Hotel_Review-g33851-d85912-Reviews-Ne

In [10]:
blinks[0][2]

u'http://www.tripadvisor.com/Hotel_Review-g33851-d1217689-Reviews-The_Study_at_Yale-New_Haven_Connecticut.html'

In [12]:
donebids = get_done_business_ids(city, engine)

In [13]:
donebids

[1217689, 85971, 225916, 119885, 119904, 83702, 85912, 83697, 119851, 85908]

In [16]:
blinks[0][1]

u'1217689'

In [17]:
int(blinks[0][1]) not in donebids

False

### skip existing entries

In [22]:
cmd = 'select biz_review_id from ta_reviews r inner join '
cmd += 'ta_hotels h on r.business_id=h.business_id '
cmd += 'where h.hotel_city = '
cmd += '"'+(' ').join(city.split('_'))+'"'
xstng_revs = [rev_id[0] for rev_id in pd.read_sql_query(cmd, engine).values]


In [24]:
#xstng_revs

### parsing utf chars

In [28]:
susrnm = ' C太郎'

In [29]:
type(susrnm)

str

In [32]:
susrnm

' C\xe5\xa4\xaa\xe9\x83\x8e'

In [34]:
susrnm.decode('utf-8', 'ignore').strip()

u'C\u592a\u90ce'

In [36]:
print(susrnm.decode('utf-8', 'ignore').strip())

C太郎


In [27]:
str(usrnm[0]).strip()

'C\xe5\xa4\xaa\xe9\x83\x8e'

In [24]:
bigdf = scrape_hotel(blinks[0][2], br, engine)

In [25]:
len(bigdf)

568

In [31]:
bigdf['business_id'] = blinks[0][0]

In [32]:
# drop the hotel_name column (this column does not exist in future versions):
bigdf.drop('hotel_name', axis=1, inplace=True)

In [34]:
bigdf.to_sql('ta_reviews', engine, if_exists='append', index=False)

In [30]:
bigdf.head(5)

,review_id,hotel_id,hotel_name,business_id,biz_review_id,biz_member_id,username,review_title,review_rating,review_text,review_date
0,NaN,NaN,NaN,NaN,310458194,87C5888145EC92BDB0F72C18EB08CAEB,PhilipNewJersey,Excellent property,5,Right down the street from the Yale campus and...,"September 15, 2015"
1,NaN,NaN,NaN,NaN,309709618,373F7ED4C06E86BE735B38FE3C5FBAE8,Gloria J,Best Place to Stay at New Haven,4,"We've stayed at a few places at New Haven, and...","September 13, 2015"
2,NaN,NaN,NaN,NaN,308615033,3846E72EAB7732919C9B65F05D9CBBE1,ILphotomom,really enjoyed my stay,5,I was visiting New Haven for work and had the ...,"September 9, 2015"
3,NaN,NaN,NaN,NaN,306265572,8868FE2D7F3A2A3E6CBDD949C6B99520,donaldk184,The Study Rocks!,5,The best place to stay in New Haven. Great roo...,"September 2, 2015"
4,NaN,NaN,NaN,NaN,305217155,10182DEB7032B071483618C256BD0261,jksingapore,Truly wonderful hotel!,5,We had a fabulous experience staying at the St...,"August 31, 2015"
